<a href="https://colab.research.google.com/github/Dragon20942/osu_pp_hypo/blob/main/OsuPP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List
import requests
import json
from bs4 import BeautifulSoup

class OsuPP:
  scores = []
  wscores = []
  hscores = []
  wpp = 0
  hwpp = 0
  pp = 0
  ppdiff = 0
  ppMaxErr = 0
  username = ''
  scorenum = 0
  hscoresAboveMin = 0

  def __init__(self):
    #@title Hypothetical Score Profile Calculator
    #@markdown #Instructions
    #@markdown ---
    #@markdown ####1. Paste the link to your osu! profile below:
    profile_link = 'https://osu.ppy.sh/users/7562902'#@param {type:"string"}
    #@markdown ####2. Select your game mode below:
    game_mode = 'osu' #@param ["osu", "taiko","fruits","mania"] {type:"string"}
    #@markdown ####3. Press ctrl + F9 to execute the code
    #@markdown ####4. Type a hypothetical score into the "New score" box
    #@markdown ####5. Click the "Insert score" button
    #@markdown ####6. Repeat steps 4-5 until you have inserted all scores
    #@markdown ####7. If you have made a mistake, click the "Clear inserted scores" button
    #@markdown ####Note: This calculator assumes that each score set is on a map not already in your top 100!

    page = requests.get(profile_link+'/'+game_mode)

    if (page.status_code != 200):
      print('Error: '+ str(page.status_code)+ ' retrieving profile\n')
      quit()

    soup = BeautifulSoup(page.content, 'html.parser')
    soup = soup.find_all("div", {"class": "js-react--profile-page osu-layout osu-layout--full"})
    soup = json.loads(soup[0]['data-initial-data'])['user']
    self.username = soup['username']
    self.pp = soup['statistics']['pp']

    profile_link = profile_link + '/scores/best?mode='+game_mode+'&limit=100&offset=0'
    pageinfo = json.loads(requests.get(profile_link).content)

    if (page.status_code != 200):
      print('Error: '+ str(page.status_code) + ' retrieving scores\n')
      quit()

    self.scorenum = len(pageinfo)
    self.scores = [i['pp'] for i in pageinfo]
    self.wscores = self.calc_weighted_pp(self.scores)
    self.wpp = round(sum(self.wscores),3)
    self.ppdiff = self.pp - self.wpp
    self.clearh()
    
    errInfSeries = 0
    if (self.scorenum == 100):
      errInfSeries = 1/(1-0.95)*0.95**99

    self.ppMaxErr = min(errInfSeries*self.scores[self.scorenum-1], self.ppdiff)

  def calc_weighted_pp(self, lin: List[float]):
    """Calculate weighted pp
    
    Args:
      lin (list): list of top play pp values, from highest to lowest pp
    Returns:
      list: list of weighted top play pp values, from highest to lowest pp
    """
    w = 1.0
    lout = lin[:]
    for i in range(0,len(lout)):
      lout[i] = lout[i]*w
      w = w*0.95
    return lout

  def clearh(self):    
    """Resets hypothetical scores and hypothetical weighted pp
    """
    self.hscores = self.scores[:]
    self.hwpp = self.pp
    self.hscoresAboveMin = 0
    errInfSeries = 0
    if (self.scorenum == 100):
      errInfSeries = 1/(1-0.95)*0.95**99
    self.ppMaxErr = min(errInfSeries*self.scores[self.scorenum-1], self.ppdiff)

  def calc_hwpp(self, v: float = 0.0):
    """Updates hypothetical scores and calculates hypothetical weighted pp if new score were set (default 0.0)

    Args:
      v (int): The hypothetical pp value to be inserted
    Returns:
      None
    """
    for i in range(0,len(self.hscores)):
      if self.hscores[i] < v:
        self.hscores.insert(i,v)
        self.hwpp = round(sum(self.calc_weighted_pp(self.hscores)) + self.ppdiff,3)
        if self.hscores[i] > self.scores[len(self.scores)-1]:
          self.hscoresAboveMin += 1
        errInfSeries = 0
        if (self.scorenum == 100):
          errInfSeries = 1/(1-0.95)*0.95**(99+self.hscoresAboveMin)
          self.ppMaxErr *= 0.95
        return None
    self.hscores.append(v)
    self.hwpp = round(sum(self.calc_weighted_pp(self.hscores)) + self.ppdiff,3)

import ipywidgets as widgets
from IPython.display import display, clear_output
import time



def insert_clicked(b):
  opp.calc_hwpp(sub.value)
  with newscorelist:
    print("New "+str(sub.value)+" pp score")
  with ppdisplay:
    clear_output()
    print((opp.username + " currently has: ") + str(opp.pp) + "pp")
    print("If " + opp.username + " sets their scores below, they would have: " + str(opp.hwpp) + "pp ±"+str(opp.ppMaxErr/2))
  with cmdout:
    print("Inserted")
    time.sleep(1)
    clear_output()


def reset_clicked(b):
  opp.clearh()
  with newscorelist:
    clear_output()
  with ppdisplay:
    clear_output()
    print((opp.username + " currently has: ") + str(opp.pp) + "pp")
    print("If " + opp.username + " sets their scores below, they would have: " + str(opp.hwpp) + "pp ±"+ str(opp.ppMaxErr/2))
  with cmdout:
    print("Cleared")
    time.sleep(1)
    clear_output()

opp = OsuPP()

insertButton = widgets.Button(description="Insert score")
resetButton = widgets.Button(description="Clear inserted scores")
cmdout = widgets.Output()
newscorelist = widgets.Output()
ppdisplay = widgets.Output()

insertButton.on_click(insert_clicked)
resetButton.on_click(reset_clicked)

buttons = widgets.HBox([insertButton,resetButton])
sub = widgets.FloatText(
    value=0.0,
    description='New score:',
    disabled=False
)

with ppdisplay:
    print(opp.username + " currently has: " + str(opp.pp) + "pp")
    print("Insert a new score by entering a pp value into the \"New score\" box and pressing the \"Insert score\" button")
widgets.VBox([buttons, sub, ppdisplay, newscorelist, cmdout])

#display(insertButton, insertOutput)
#display(calcButton, calcOutput)
#display(resetButton, resetOutput)
